In [1]:
# This notebook is based on MNE-Python tutorials and examples, e.g.
# https://mne.tools/stable/auto_tutorials/forward/20_source_alignment.html
# https://mne.tools/stable/auto_tutorials/forward/25_automated_coreg.html

# Modified by Olaf Hauk, olaf.hauk@mrc-cbu.cam.ac.uk
# These notebooks complement the online lectures "Introduction to EEG/MEG analysis":
# https://www.youtube.com/playlist?list=PLp67eqWCj2f_DBsCMkIOBpBbLWGAUKtu5

In [2]:
import nibabel as nib
import numpy as np
from scipy import linalg

import mne
from mne.io.constants import FIFF

data_path = mne.datasets.sample.data_path()
subjects_dir = data_path / "subjects"
raw_fname = data_path / "MEG" / "sample" / "sample_audvis_raw.fif"
trans_fname = data_path / "MEG" / "sample" / "sample_audvis_raw-trans.fif"
raw = mne.io.read_raw_fif(raw_fname)
trans = mne.read_trans(trans_fname)
src = mne.read_source_spaces(subjects_dir / "sample" / "bem" / "sample-oct-6-src.fif")

# Load the T1 file and change the header information to the correct units
t1w = nib.load(data_path / "subjects" / "sample" / "mri" / "T1.mgz")
t1w = nib.Nifti1Image(t1w.dataobj, t1w.affine)
t1w.header["xyzt_units"] = np.array(10, dtype="uint8")
t1_mgh = nib.MGHImage(t1w.dataobj, t1w.affine)

Opening raw data file C:\Users\olaf\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read


.. raw:: html

   <style>
   .pink {color:DarkSalmon; font-weight:bold}
   .blue {color:DeepSkyBlue; font-weight:bold}
   .gray {color:Gray; font-weight:bold}
   .magenta {color:Magenta; font-weight:bold}
   .purple {color:Indigo; font-weight:bold}
   .green {color:LimeGreen; font-weight:bold}
   .red {color:Red; font-weight:bold}
   </style>

.. role:: pink
.. role:: blue
.. role:: gray
.. role:: magenta
.. role:: purple
.. role:: green
.. role:: red


## Understanding coordinate frames
For M/EEG source imaging, there are three **coordinate frames** must be
brought into alignment using two 3D [transformation matrices](wiki_xform_)
that define how to rotate and translate points in one coordinate frame
to their equivalent locations in another. The three main coordinate frames
are:

* :blue:`"meg"`: the coordinate frame for the physical locations of MEG sensors
* :gray:`"mri"`: the coordinate frame for MRI images, and scalp/skull/brain
  surfaces derived from the MRI images
* :pink:`"head"`: the coordinate frame for digitized sensor locations and
  scalp landmarks ("fiducials")


Each of these are described in more detail in the next section.

A good way to start visualizing these coordinate frames is to use the
`mne.viz.plot_alignment` function, which is used for creating or inspecting
the transformations that bring these coordinate frames into alignment, and
displaying the resulting alignment of EEG sensors, MEG sensors, brain
sources, and conductor models. If you provide ``subjects_dir`` and
``subject`` parameters, the function automatically loads the subject's
Freesurfer MRI surfaces. Important for our purposes, passing
``show_axes=True`` to `~mne.viz.plot_alignment` will draw the origin of each
coordinate frame in a different color, with axes indicated by different sized
arrows:

* shortest arrow: (**R**)ight / X
* medium arrow: forward / (**A**)nterior / Y
* longest arrow: up / (**S**)uperior / Z

Note that all three coordinate systems are **RAS** coordinate frames and
hence are also `right-handed`_ coordinate systems. Finally, note that the
``coord_frame`` parameter sets which coordinate frame the camera
should initially be aligned with. Let's have a look:



In [3]:
fig = mne.viz.plot_alignment(
    raw.info,
    trans=trans,
    subject="sample",
    subjects_dir=subjects_dir,
    surfaces="head-dense",
    show_axes=True,
    dig=True,
    eeg=[],
    meg="sensors",
    coord_frame="meg",
    mri_fiducials="estimated",
)
mne.viz.set_3d_view(fig, 45, 90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
print(
    "Distance from head origin to MEG origin: %0.1f mm"
    % (1000 * np.linalg.norm(raw.info["dev_head_t"]["trans"][:3, 3]))
)
print(
    "Distance from head origin to MRI origin: %0.1f mm"
    % (1000 * np.linalg.norm(trans["trans"][:3, 3]))
)
dists = mne.dig_mri_distances(raw.info, trans, "sample", subjects_dir=subjects_dir)
print(
    f"Distance from {len(dists)} digitized points to head surface: "
    f"{1000 * np.mean(dists):0.1f} mm"
)

Using pyvistaqt 3d backend.
False
Using lh.seghead for head surface.
Channel types::	grad: 203, mag: 102
Distance from head origin to MEG origin: 65.0 mm
Distance from head origin to MRI origin: 29.9 mm
Using surface from C:\Users\olaf\mne_data\MNE-sample-data\subjects\sample\bem\sample-head.fif.
Distance from 72 digitized points to head surface: 1.7 mm


### Coordinate frame definitions
1. Neuromag/Elekta/MEGIN head coordinate frame ("head", :pink:`pink axes`)
     The head coordinate frame is defined through the coordinates of
     anatomical landmarks on the subject's head: usually the Nasion (`NAS`_),
     and the left and right preauricular points (`LPA`_ and `RPA`_).
     Different MEG manufacturers may have different definitions of the head
     coordinate frame. A good overview can be seen in the
     `FieldTrip FAQ on coordinate systems`_.

     For Neuromag/Elekta/MEGIN, the head coordinate frame is defined by the
     intersection of

     1. the line between the LPA (:red:`red sphere`) and RPA
        (:purple:`purple sphere`), and
     2. the line perpendicular to this LPA-RPA line one that goes through
        the Nasion (:green:`green sphere`).

     The axes are oriented as **X** origin→RPA, **Y** origin→NAS,
     **Z** origin→upward (orthogonal to X and Y).

     .. note:: The required 3D coordinates for defining the head coordinate
               frame (NAS, LPA, RPA) are measured at a stage separate from
               the MEG data recording. There exist numerous devices to
               perform such measurements, usually called "digitizers". For
               example, see the devices by the company `Polhemus`_.

2. MEG device coordinate frame ("meg", :blue:`blue axes`)
     The MEG device coordinate frame is defined by the respective MEG
     manufacturers. All MEG data is acquired with respect to this coordinate
     frame. To account for the anatomy and position of the subject's head, we
     use so-called head position indicator (HPI) coils. The HPI coils are
     placed at known locations on the scalp of the subject and emit
     high-frequency magnetic fields used to coregister the head coordinate
     frame with the device coordinate frame.

     From the Neuromag/Elekta/MEGIN user manual:

         The origin of the device coordinate system is located at the center
         of the posterior spherical section of the helmet with X axis going
         from left to right and Y axis pointing front. The Z axis is, again
         normal to the plane with positive direction up.

     .. note:: The HPI coils are shown as :magenta:`magenta spheres`.
               Coregistration happens at the beginning of the recording and
               the head↔meg transformation matrix is stored in
               ``raw.info['dev_head_t']``.

3. MRI coordinate frame ("mri", :gray:`gray axes`)
     Defined by Freesurfer, the "MRI surface RAS" coordinate frame has its
     origin at the center of a 256×256×256 1mm anisotropic volume (though the
     center may not correspond to the anatomical center of the subject's
     head).

     .. note:: We typically align the MRI coordinate frame to the head
               coordinate frame through a [rotation and translation matrix](wiki_xform_), that we refer to in MNE as ``trans``.

### A bad example
Let's try using `~mne.viz.plot_alignment` by making ``trans`` the identity
transform. This (incorrectly!) equates the MRI and head coordinate frames.



In [5]:
identity_trans = mne.transforms.Transform("head", "mri")
mne.viz.plot_alignment(
    raw.info,
    trans=identity_trans,
    subject="sample",
    src=src,
    subjects_dir=subjects_dir,
    dig=True,
    surfaces=["head-dense", "white"],
    coord_frame="meg",
)

False
Using lh.seghead for head surface.
Getting helmet for system 306m
Channel types::	grad: 203, mag: 102, eeg: 59


### A good example
Here is the same plot, this time with the ``trans`` properly defined
(using a precomputed transformation matrix).



In [6]:
mne.viz.plot_alignment(
    raw.info,
    trans=trans,
    subject="sample",
    src=src,
    subjects_dir=subjects_dir,
    dig=True,
    surfaces=["head-dense", "white"],
    coord_frame="meg",
)

False
Using lh.seghead for head surface.
Getting helmet for system 306m
Channel types::	grad: 203, mag: 102, eeg: 59



## Defining the head↔MRI ``trans`` using the GUI
You can try creating the head↔MRI transform yourself using
:func:`mne.gui.coregistration`.

* To set the MRI fiducials, make sure ``Lock Fiducials`` is toggled off.
* Set the landmarks by clicking the radio button (LPA, Nasion, RPA) and then
  clicking the corresponding point in the image.

<div class="alert alert-info"><h4>Note</h4><p>The position of each fiducial used is the center of the octahedron icon.</p></div>

* After doing this for all the landmarks, toggle ``Lock Fiducials`` radio
  button and optionally pressing ``Save MRI Fid.`` which will save to a
  default location in the ``bem`` folder of the Freesurfer subject directory.
* Then you can load the digitization data from the raw file
  (``Path to info``).
* Click ``Fit ICP``. This will align the digitization points to the
  head surface. Sometimes the fitting algorithm doesn't find the correct
  alignment immediately. You can try first fitting using LPA/RPA or fiducials
  and then align according to the digitization. You can also finetune
  manually with the controls on the right side of the panel.
* Click ``Save`` (lower right corner of the panel), set the filename
  and read it with :func:`mne.read_trans`.

For more information, see this video:

.. youtube:: ALV5qqMHLlQ

<div class="alert alert-info"><h4>Note</h4><p>Coregistration can also be automated as shown in `tut-auto-coreg`.</p></div>



In [7]:
# Start the GUI for co-registration
mne.gui.coregistration(subject="sample", subjects_dir=subjects_dir)

False
    Triangle neighbors and vertex normals...
Using high resolution head model in C:\Users\olaf\mne_data\MNE-sample-data\subjects\sample\surf\lh.seghead
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\olaf\mne_data\MNE-sample-data\subjects\sample\bem\sample-fiducials.fif.
Using high resolution head model in C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif.
Loading MRI fiducials from C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif... Done!
    Triangle neighbors and vertex normals...
Using high resolution head model in C:\Users\olaf\mne_data\MNE-sample-data\subjects\sample\surf\lh.seghead
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\olaf\mne_data\MNE-sample-data\subjects\sample\bem\sample-fiducials.

It is also possible to use :func:`mne.gui.coregistration`
to warp a subject (usually ``fsaverage``) to subject digitization data. Note that you can use the "MRI Scaling" options to (non-)linearly rescale the transformation (see
[these slides](https://www.slideshare.net/mne-python/mnepython-scale-mri)).




In [8]:
# use Freesurfer's fsaverage brain for co-registration
mne.gui.coregistration(subject="fsaverage", subjects_dir=subjects_dir)

False
Using high resolution head model in C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif.
Using high resolution head model in C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif.
Loading MRI fiducials from C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif... Done!
Using fsaverage-head-dense.fif for head surface.
    1 BEM surfaces found
    Reading a surface...
[done]
    1 BEM surfaces read
Loading MRI fiducials from C:\Users\olaf\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif... Done!
Using fsaverage-head-dense.fif for head surface.
    1 BEM surface